## Decision Trees

- Simple Tree like structure, model makes a decision at every node
- Useful in simple tasks
- One of the most popular algorithm
- Easy explainability, easy to show how a decision process works!

### Why decision trees are popular?
   - Easy to interpret and present
   - Well defined Logic, mimic human level thought
   - Random Forests, Ensembles of decision trees are more powerful classifiers
   - Feature values are preferred to be **categorical**. If the values are continuous then they are discretized prior to building the model.

##  Build Decision Trees

Two common algorithms - 

- CART (Classification and Regression Trees) → uses Gini Index(Classification) as metric.
- ID3 (Iterative Dichotomiser 3) → uses Entropy function and Information gain as metrics


    


### Let's Construct the Decision Tree
#### ID3 Implementation

In [13]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv("mushrooms.csv")

In [6]:
data.head(n=5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [191]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data = data.apply(le.fit_transform)

In [192]:
X, y = data.drop(["class"], axis=1), data["class"].values

In [193]:
class DecisionTree:
    
    def __init__(self, max_depth = 1, min_data_split = 50):
        self.max_depth = max_depth
        self.min_data_split = min_data_split
    
    def fit(self, X, y):
        self.root = self.construct_node(X, y, 0)
    
    
    def traverse_node(self, node, X):
        if node["is_leaf"]:
            return node["predictions"]

        feat = node["column_selected"]
        feat_val = X[feat]
        return self.traverse_node(node["childrens"][feat_val], X)
        
    def predict(self, X):
        predictions = []
        for i in range(len(X)):
            probs, classes = self.traverse_node(self.root, X.iloc[i])
            predictions.append(classes[np.argmax(probs)])
        return predictions
    
    def construct_node(self, X, y, cur_depth):
        
        if len(X) == 0  or len(X.columns) == 0:
            return {}
        if cur_depth > self.max_depth:
            return {}
        node = {"column_selected": None,
                    "childrens": {}, "is_leaf": False}
        if len(X) >= self.min_data_split:
            before_split_entropy = self.entropy(y)

            weighted_after_split_entropy = np.zeros((len(X.columns),))

            for i, feat in enumerate(X.columns):
                feat_vals = X[feat].unique()
                for feat_val in feat_vals:
                    weighted_after_split_entropy[i] += (X[feat] == feat_val).mean()*self.entropy(y[X[feat] == feat_val])

            node["column_selected"] = X.columns[np.argmax(before_split_entropy -  weighted_after_split_entropy)]

            feat = node["column_selected"]

            feat_vals = X[feat].unique()
            for feat_val in feat_vals:
                X_split = X[X[feat] == feat_val].drop([feat], axis=1)
                y_split = y[X[feat] == feat_val]
                child_node = self.construct_node(X_split, y_split, cur_depth + 1)
                if child_node == {}:
                    node["is_leaf"] = True
                    node["childrens"] = []
                    break
                node["childrens"][feat_val] = child_node
        else:
            node["is_leaf"] = True
        if node["is_leaf"]:
            node["predictions"] = self.calc_class_prob(y)
        return node
    
    def calc_class_prob(self, y):
        classes = np.unique(y)
        probs = np.empty((classes.shape[0],), dtype=np.float32)

        for i, cls in enumerate(classes):
            probs[i] = (y == cls).mean()
            
        return probs, classes
    
    def entropy(self, y):
        probs, classes = self.calc_class_prob(y)
        return np.sum(-1*probs*np.log2(probs))

In [194]:
dt = DecisionTree()

In [195]:
X_test, y_test = X.iloc[-100:], y[-100:]
X_train, y_train = X.iloc[:-100], y[:-100]

In [196]:
dt.fit(X_train.drop(["odor"], axis=1), y_train)

In [204]:
y_pred = dt.predict(X_test)

In [209]:
(y_pred == y_test).mean()

0.8685194416749751

### Sklearn Decision Trees

In [179]:
from sklearn.tree import DecisionTreeClassifier 

In [182]:
sk_dt = DecisionTreeClassifier(criterion="entropy", max_depth=5, min_samples_split=50)

In [199]:
sk_dt.fit(X_train.drop(["odor"], axis=1), y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=50)

In [202]:
sk_y_pred = sk_dt.predict(X_test.drop(["odor"], axis=1))

In [203]:
(sk_y_pred == y_test).mean()

0.99

### Random Forest

In [211]:
from sklearn.ensemble import RandomForestClassifier

In [220]:
rf = RandomForestClassifier(n_estimators=10, max_depth=3)

In [221]:
rf.fit(X_train.drop(["odor"], axis=1), y_train)

RandomForestClassifier(max_depth=3, n_estimators=10)

In [222]:
rf.score(X_train.drop(["odor"], axis=1), y_train)

0.9479062811565304

In [223]:
rf.score(X_test.drop(["odor"], axis=1), y_test)

1.0